In [1]:
import sys
import os

PROJECT_DIR = '/home/evgenshuben/Desktop/gitReps/YandexCup/'
sys.path.append(PROJECT_DIR)
os.chdir(PROJECT_DIR)

In [49]:
from omegaconf import DictConfig, OmegaConf
import hydra
from hydra import initialize, compose, initialize_config_dir
from hydra.utils import instantiate

import torch.nn as nn
import torch
import torch.nn.functional as F
import pandas as pd
import numpy as np
from tqdm import tqdm
import faiss
from sklearn.model_selection import train_test_split
import random

In [3]:
%load_ext autoreload
%autoreload 2

from scr.train import Trainer
from scr.data.dataset import CoverDatasetClassifier

# Load data

In [64]:
nearest_track_idx_pd = pd.read_parquet('/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/reranker/train_dataset_pd.pq')

In [65]:
nearest_track_idx_pd.head()

,track_id,neighbors,negatives,positives
0,343223,"[343223, 314991, 254137, 200291, 101812, 12627...","[314991, 254137, 200291, 101812, 126276, 35797...","[307473, 361210, 101132, 343223]"
1,361210,"[361210, 101132, 307473, 343223, 281680, 33874...","[281680, 338741, 204228, 182103, 189128, 23351...","[59234, 101132, 307473, 343223, 361210]"
2,114472,"[114472, 247079, 109255, 216454, 192349, 13474...","[109255, 192349, 258956, 236469, 221151, 22684...","[59234, 216454, 247079, 114472, 215467, 307473..."
3,134744,"[134744, 215467, 258956, 247079, 179511, 30816...","[258956, 179511, 308169, 146032, 136942, 29370...","[59234, 216454, 247079, 114472, 215467, 101132..."
4,271362,"[271362, 315298, 144962, 332689, 180183, 19886...","[315298, 144962, 332689, 180183, 198867, 29852...",[271362]


In [66]:
def remove_track_id_from_positives(row):
    track_id = row['track_id']
    positives = row['positives']
    return [pos for pos in positives if pos != track_id]


In [67]:
nearest_track_idx_pd['positives'] = nearest_track_idx_pd.apply(remove_track_id_from_positives, axis=1)

In [68]:
def sample_positives(row):
    positives = row['positives']
    
    if isinstance(positives, str):
        positives = eval(positives)
    return random.sample(positives, min(3, len(positives)))

In [69]:
def sample_negatives(row):
    negatives = row['negatives']
    
    if isinstance(negatives, str):
        try:
            negatives = eval(negatives)
        except:
            negatives = []
    
    if isinstance(negatives, np.ndarray):
        negatives = negatives.tolist() 
    elif not isinstance(negatives, list):
        negatives = list(negatives) if negatives else []

    return random.sample(negatives, min(10, len(negatives)))

In [70]:
# Применяем функцию к каждому ряду
nearest_track_idx_pd['positives_sample'] = nearest_track_idx_pd.apply(sample_positives, axis=1)
nearest_track_idx_pd['negatives_sample'] = nearest_track_idx_pd.apply(sample_negatives, axis=1)

In [71]:
nearest_track_idx_pd.head()

,track_id,neighbors,negatives,positives,positives_sample,negatives_sample
0,343223,"[343223, 314991, 254137, 200291, 101812, 12627...","[314991, 254137, 200291, 101812, 126276, 35797...","[307473, 361210, 101132]","[101132, 361210, 307473]","[129739, 314382, 305229, 177213, 266728, 32334..."
1,361210,"[361210, 101132, 307473, 343223, 281680, 33874...","[281680, 338741, 204228, 182103, 189128, 23351...","[59234, 101132, 307473, 343223]","[307473, 343223, 59234]","[316818, 154533, 138248, 214362, 142652, 19425..."
2,114472,"[114472, 247079, 109255, 216454, 192349, 13474...","[109255, 192349, 258956, 236469, 221151, 22684...","[59234, 216454, 247079, 215467, 307473, 134744]","[215467, 216454, 134744]","[340036, 344253, 279723, 244602, 192349, 12253..."
3,134744,"[134744, 215467, 258956, 247079, 179511, 30816...","[258956, 179511, 308169, 146032, 136942, 29370...","[59234, 216454, 247079, 114472, 215467, 101132...","[114472, 247079, 216454]","[109356, 231732, 258956, 190380, 119905, 33590..."
4,271362,"[271362, 315298, 144962, 332689, 180183, 19886...","[315298, 144962, 332689, 180183, 198867, 29852...",[],[],"[332689, 101597, 351652, 85313, 176303, 228944..."


In [73]:
nearest_track_idx_pd[nearest_track_idx_pd['positives_sample'].str.len() == 0].shape

(35996, 6)

In [74]:
nearest_track_idx_pd[nearest_track_idx_pd['negatives_sample'].str.len() == 0].shape

(958, 6)

Уберем нафиг клики без позитивов и негативов

In [81]:
full_dataset_pd = nearest_track_idx_pd[nearest_track_idx_pd['positives_sample'].str.len() != 0]
full_dataset_pd = full_dataset_pd[full_dataset_pd['negatives_sample'].str.len() == 10]
full_dataset_pd = full_dataset_pd[full_dataset_pd['positives'].str.len() <= 10]

full_dataset_pd.shape

(142820, 6)

In [82]:
full_dataset_pd[['track_id', 'positives_sample', 'negatives_sample']].head()

,track_id,positives_sample,negatives_sample
0,343223,"[101132, 361210, 307473]","[129739, 314382, 305229, 177213, 266728, 32334..."
1,361210,"[307473, 343223, 59234]","[316818, 154533, 138248, 214362, 142652, 19425..."
2,114472,"[215467, 216454, 134744]","[340036, 344253, 279723, 244602, 192349, 12253..."
3,134744,"[114472, 247079, 216454]","[109356, 231732, 258956, 190380, 119905, 33590..."
5,307473,"[343223, 215467, 247079]","[191559, 96854, 117288, 242661, 282390, 132922..."


### Трейн датасет

Тут надо насемплировать 0 и 1 для каждого трека (для валидации оставляем всех кандидатов)

In [218]:
def split_train_val(df, test_size=0.1, random_state=42):  
    unique_track_ids = df['track_id'].unique()
    train_ids, val_ids = train_test_split(unique_track_ids, test_size=test_size, random_state=random_state)
    
    train_df = df[df['track_id'].isin(train_ids)].reset_index(drop=True)
    val_df = df[df['track_id'].isin(val_ids)].reset_index(drop=True)
    return train_df, val_df

In [219]:
df_sampled = full_dataset_pd.sample(n=3000, random_state=42)
df_sampled.shape

(3000, 6)

In [226]:
train_df, val_df = split_train_val(df_sampled, test_size=0.05)

In [227]:
train_df = (
    train_df[['track_id', 'positives_sample', 'negatives_sample']]
    .rename(columns={
        'positives_sample': 'positives',
        'negatives_sample': 'negatives'
    })
)
val_df = val_df[['track_id', 'negatives', 'positives']]

In [228]:
train_df.head()

,track_id,positives,negatives
0,124695,[296609],"[80938, 361675, 270840, 196872, 214522, 176676..."
1,234661,[185803],"[265562, 309517, 353870, 96985, 211152, 99728,..."
2,364209,"[112120, 202129, 216016]","[352998, 195842, 278694, 225159, 246869, 98303..."
3,189362,"[83281, 160008, 69935]","[103296, 285115, 59645, 353525, 182968, 329941..."
4,111922,"[73853, 286597, 173279]","[330832, 142650, 115469, 284189, 293017, 96271..."


In [229]:
val_df.head()

,track_id,negatives,positives
0,74032,"[335131, 135256, 252591, 96485, 180004, 207765...","[237563, 284629]"
1,229680,"[289701, 126612, 62403, 61746, 251567, 354258,...","[125795, 300998, 245862, 226545]"
2,139821,"[296149, 64918, 65236, 100210, 86435, 174518, ...",[132085]
3,60048,"[267750, 334166, 322224, 107092, 190052, 17920...",[221901]
4,148838,"[198056, 229740, 214632, 160667, 226510, 14734...","[208200, 70827, 305138, 75926]"


In [230]:
def flatten_candidates(df):
    # Преобразуем строки в списки, если это необходимо
    df['negatives'] = df['negatives'].apply(lambda x: eval(x) if isinstance(x, str) else x)
    df['positives'] = df['positives'].apply(lambda x: eval(x) if isinstance(x, str) else x)
    
    # Добавляем столбец target и объединяем кандидатов
    df_negatives = df[['track_id', 'negatives']].copy()
    df_negatives['target'] = 0  # Метка для негативов
    df_negatives = df_negatives.explode('negatives').rename(columns={'negatives': 'candidate'})

    df_positives = df[['track_id', 'positives']].copy()
    df_positives['target'] = 1  # Метка для позитивов
    df_positives = df_positives.explode('positives').rename(columns={'positives': 'candidate'})
    
    # Объединяем позитивные и негативные кандидаты в один DataFrame
    flat_df = pd.concat([df_negatives, df_positives], ignore_index=True)
    
    return flat_df


In [231]:
train_flat_pd = flatten_candidates(train_df)
val_flat_pd = flatten_candidates(val_df)


print('train shape:', train_flat_pd.shape)
print('val shape:', val_flat_pd.shape)

train shape: (35058, 3)
val shape: (29850, 3)


###### inferense model

теперь осталось проставлять предикты от модели к датасетам

Для этого нужно загрузить модель и заинференсить ее на парах, которые получились



In [143]:
train_flat_pd

,track_id,candidate,target
0,74032,113323,0
1,74032,85118,0
2,74032,176518,0
3,74032,165385,0
4,74032,57577,0
...,...,...,...
57132,111486,58924,1
57133,111486,100131,1
57134,306114,357645,1
57135,306114,310477,1


### Try inference models

### 15

In [144]:
# Путь к конфигурационному файлу, сохраненному Hydra
config_path = "/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-15/hydra/.hydra/config.yaml"
cfg = OmegaConf.load(config_path)
# cfg.enviroment.device = 'cpu'
print(cfg.model.backbone_name)
print(cfg.pipeline.model_ckpt)

hgnetv2_b4.ssld_stage2_ft_in1k
/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-15/data/model/best-model-epoch=1-max_secs=50.pt


In [164]:
from sklearn.metrics.pairwise import cosine_similarity


In [176]:
trainer.cfg.enviroment.device

{'progress_bar': True, 'device': 'cuda', 'num_channels': 1}

In [215]:
def get_embedding(track_id, model):
    cqt = trainer.train_dataloader.dataset._load_cqt(track_id)
    img = transforms_pipeline(cqt).to(trainer.cfg.enviroment.device)
    emb = model(img.unsqueeze(0))
    return emb['emb'].squeeze(0)

In [232]:
def calculate_scores(df, model, output_column='score'):
    scores = []
    for _, row in tqdm(df.iterrows(), total=df.shape[0]):
        track_id = row['track_id']
        candidate = row['candidate']
        
        # Получаем эмбеддинги для track_id и candidate
        track_embedding = get_embedding(track_id, model).cpu().detach().numpy()
        candidate_embedding = get_embedding(candidate, model).cpu().detach().numpy()
        
        # Считаем косинусное сходство
        score = cosine_similarity([track_embedding], [candidate_embedding])[0, 0]
        scores.append(score)
    
    # Добавляем столбец со скорами в DataFrame
    df[output_column] = scores
    return df

In [198]:
trainer = Trainer(cfg)

/home/evgenshuben/Desktop/gitReps/YandexCup/scr/train.py:51: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.load(self.cfg.pipeline.model_ckpt

In [206]:
# trainer.model.eval()

In [212]:
# trainer.model

In [233]:
calculate_scores(train_flat_pd, trainer.model, output_column='15')

100%|██████████| 35058/35058 [06:10<00:00, 94.74it/s] 


,track_id,candidate,target,15
0,124695,80938,0,0.308677
1,124695,361675,0,0.342522
2,124695,270840,0,0.333195
3,124695,196872,0,0.275479
4,124695,214522,0,0.275690
...,...,...,...,...
35053,249054,180251,1,0.360768
35054,366718,220251,1,0.386072
35055,366718,326755,1,0.436662
35056,366718,87450,1,0.387213


In [236]:
calculate_scores(val_flat_pd, trainer.model, output_column='15')

100%|██████████| 29850/29850 [05:13<00:00, 95.25it/s] 


,track_id,candidate,target,15
0,74032,335131,0,0.515230
1,74032,135256,0,0.530595
2,74032,252591,0,0.449715
3,74032,96485,0,0.430516
4,74032,180004,0,0.434247
...,...,...,...,...
29845,71047,199792,1,0.344137
29846,224189,283204,1,0.528731
29847,224189,171014,1,0.471906
29848,224189,298607,1,0.653968


###### 17

In [238]:
# Путь к конфигурационному файлу, сохраненному Hydra
config_path = "/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-17/hydra/.hydra/config.yaml"
cfg = OmegaConf.load(config_path)
# cfg.enviroment.device = 'cpu'
print(cfg.model.backbone_name)
print(cfg.pipeline.model_ckpt)

trainer = Trainer(cfg)
trainer.model.eval()

edgenext_base.in21k_ft_in1k
/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-17/data/model/best-model-epoch=1-max_secs=50.pt


/home/evgenshuben/Desktop/gitReps/YandexCup/scr/train.py:51: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.load(self.cfg.pipeline.model_ckpt

BackBone(
  (model): EdgeNeXt(
    (stem): Sequential(
      (0): Conv2d(3, 80, kernel_size=(4, 4), stride=(4, 4))
      (1): LayerNorm2d((80,), eps=1e-06, elementwise_affine=True)
    )
    (stages): Sequential(
      (0): EdgeNeXtStage(
        (downsample): Identity()
        (blocks): Sequential(
          (0): ConvBlock(
            (conv_dw): Conv2d(80, 80, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=80)
            (norm): LayerNorm((80,), eps=1e-06, elementwise_affine=True)
            (mlp): Mlp(
              (fc1): Linear(in_features=80, out_features=320, bias=True)
              (act): GELU(approximate='none')
              (drop1): Dropout(p=0.0, inplace=False)
              (norm): Identity()
              (fc2): Linear(in_features=320, out_features=80, bias=True)
              (drop2): Dropout(p=0.0, inplace=False)
            )
            (drop_path): Identity()
          )
          (1): ConvBlock(
            (conv_dw): Conv2d(80, 80, kernel_size=(3, 3)

In [239]:
calculate_scores(train_flat_pd, trainer.model, output_column='17')

100%|██████████| 35058/35058 [06:24<00:00, 91.17it/s]


,track_id,candidate,target,15,17
0,124695,80938,0,0.308677,0.236296
1,124695,361675,0,0.342522,0.193063
2,124695,270840,0,0.333195,0.390681
3,124695,196872,0,0.275479,0.321069
4,124695,214522,0,0.275690,0.335157
...,...,...,...,...,...
35053,249054,180251,1,0.360768,0.329655
35054,366718,220251,1,0.386072,0.457326
35055,366718,326755,1,0.436662,0.406042
35056,366718,87450,1,0.387213,0.227449


In [240]:
calculate_scores(val_flat_pd, trainer.model, output_column='17')

100%|██████████| 29850/29850 [05:31<00:00, 90.05it/s]


,track_id,candidate,target,15,17
0,74032,335131,0,0.515230,0.474778
1,74032,135256,0,0.530595,0.522620
2,74032,252591,0,0.449715,0.546870
3,74032,96485,0,0.430516,0.425306
4,74032,180004,0,0.434247,0.406036
...,...,...,...,...,...
29845,71047,199792,1,0.344137,0.391586
29846,224189,283204,1,0.528731,0.436251
29847,224189,171014,1,0.471906,0.513847
29848,224189,298607,1,0.653968,0.586536


###### 18

In [244]:
# Путь к конфигурационному файлу, сохраненному Hydra
config_path = "/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-18/hydra/.hydra/config.yaml"
cfg = OmegaConf.load(config_path)
# cfg.enviroment.device = 'cpu'
print(cfg.model.backbone_name)
print(cfg.pipeline.model_ckpt)

trainer = Trainer(cfg)
trainer.model.eval()

tf_efficientnet_b4.ns_jft_in1k
/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-18/data/model/best-model-epoch=0-max_secs=50.pt


/home/evgenshuben/Desktop/gitReps/YandexCup/scr/train.py:51: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.load(self.cfg.pipeline.model_ckpt

BackBone(
  (model): EfficientNet(
    (conv_stem): Conv2dSame(3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn1): BatchNormAct2d(
      48, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): SiLU(inplace=True)
    )
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
          (bn1): BatchNormAct2d(
            48, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): SiLU(inplace=True)
          )
          (aa): Identity()
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
            (act1): SiLU(inplace=True)
            (conv_expand): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
            (gate): Sigmoid()
          )
          (conv_pw): Conv2d(48, 24, ke

In [245]:
calculate_scores(train_flat_pd, trainer.model, output_column='18')
calculate_scores(val_flat_pd, trainer.model, output_column='18')

100%|██████████| 29850/29850 [09:18<00:00, 53.49it/s]


,track_id,candidate,target,15,17,18
0,74032,335131,0,0.515230,0.474778,0.512419
1,74032,135256,0,0.530595,0.522620,0.533862
2,74032,252591,0,0.449715,0.546870,0.445078
3,74032,96485,0,0.430516,0.425306,0.447900
4,74032,180004,0,0.434247,0.406036,0.410062
...,...,...,...,...,...,...
29845,71047,199792,1,0.344137,0.391586,0.335503
29846,224189,283204,1,0.528731,0.436251,0.479341
29847,224189,171014,1,0.471906,0.513847,0.468593
29848,224189,298607,1,0.653968,0.586536,0.592122


###### 21

In [246]:
# Путь к конфигурационному файлу, сохраненному Hydra
config_path = "/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-21/hydra/.hydra/config.yaml"
cfg = OmegaConf.load(config_path)
# cfg.enviroment.device = 'cpu'
print(cfg.model.backbone_name)
print(cfg.pipeline.model_ckpt)

trainer = Trainer(cfg)
trainer.model.eval()

calculate_scores(train_flat_pd, trainer.model, output_column='21')
calculate_scores(val_flat_pd, trainer.model, output_column='21')

rexnetr_300.sw_in12k_ft_in1k
/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-21/data/model/best-model-epoch=0-max_secs=50.pt


/home/evgenshuben/Desktop/gitReps/YandexCup/scr/train.py:51: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.load(self.cfg.pipeline.model_ckpt

,track_id,candidate,target,15,17,18,21
0,74032,335131,0,0.515230,0.474778,0.512419,0.395669
1,74032,135256,0,0.530595,0.522620,0.533862,0.411927
2,74032,252591,0,0.449715,0.546870,0.445078,0.468937
3,74032,96485,0,0.430516,0.425306,0.447900,0.351324
4,74032,180004,0,0.434247,0.406036,0.410062,0.318882
...,...,...,...,...,...,...,...
29845,71047,199792,1,0.344137,0.391586,0.335503,0.308169
29846,224189,283204,1,0.528731,0.436251,0.479341,0.419670
29847,224189,171014,1,0.471906,0.513847,0.468593,0.348787
29848,224189,298607,1,0.653968,0.586536,0.592122,0.517904


###### 22

In [247]:
# Путь к конфигурационному файлу, сохраненному Hydra
config_path = "/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-22/hydra/.hydra/config.yaml"
cfg = OmegaConf.load(config_path)
# cfg.enviroment.device = 'cpu'
print(cfg.model.backbone_name)
print(cfg.pipeline.model_ckpt)

trainer = Trainer(cfg)
trainer.model.eval()

calculate_scores(train_flat_pd, trainer.model, output_column='22')
calculate_scores(val_flat_pd, trainer.model, output_column='22')

efficientnet_b5.sw_in12k_ft_in1k
/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-22/data/model/best-model-epoch=0-max_secs=50.pt


/home/evgenshuben/Desktop/gitReps/YandexCup/scr/train.py:51: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.load(self.cfg.pipeline.model_ckpt

,track_id,candidate,target,15,17,18,21,22
0,74032,335131,0,0.515230,0.474778,0.512419,0.395669,0.388563
1,74032,135256,0,0.530595,0.522620,0.533862,0.411927,0.421486
2,74032,252591,0,0.449715,0.546870,0.445078,0.468937,0.623570
3,74032,96485,0,0.430516,0.425306,0.447900,0.351324,0.249243
4,74032,180004,0,0.434247,0.406036,0.410062,0.318882,0.281233
...,...,...,...,...,...,...,...,...
29845,71047,199792,1,0.344137,0.391586,0.335503,0.308169,0.335632
29846,224189,283204,1,0.528731,0.436251,0.479341,0.419670,0.585848
29847,224189,171014,1,0.471906,0.513847,0.468593,0.348787,0.367855
29848,224189,298607,1,0.653968,0.586536,0.592122,0.517904,0.551696


### train Reranker